### Content Based Recommendations

In the previous notebook, you were introduced to a way of making recommendations using collaborative filtering.  However, using this technique there are a large number of users who were left without any recommendations at all.  Other users were left with fewer than the ten recommendations that were set up by our function to retrieve....

In order to help these users out, let's try another technique: **content based** recommendations. Let's start off where we were in the previous notebook.

In [1]:
! pip install progressbar

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from IPython.display import HTML
import progressbar
import tests as t
import pickle

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']


all_recs = pickle.load(open("all_recs.p", "rb"))

### Datasets

From the above, you now have access to three important items that you will be using throughout the rest of this notebook.  

`a.` **movies** - a dataframe of all of the movies in the dataset along with other content related information about the movies (genre and date)


`b.` **reviews** - this was the main dataframe used before for collaborative filtering, as it contains all of the interactions between users and movies.


`c.` **all_recs** - a dictionary where each key is a user, and the value is a list of movie recommendations based on collaborative filtering

For the individuals in **all_recs** who did receive 10 recommendations using collaborative filtering, we don't really need to worry about them.  However, there were a number of individuals in our dataset who did not receive any recommendations.

-----

`1.` Let's start with finding all of the users in our dataset who didn't get all 10 ratings we would have liked them to have using collaborative filtering.  

In [3]:
users_with_all_recs = [] # Store user ids who have all their recommendations in this (10 or more)
users_with_part_recs = [] # Store user ids who have some but less than 10 recommendations
users_who_need_recs = [] # Store users who still need recommendations here

for user, movie_recs in all_recs.items():
    if len(movie_recs) >= 10:
        users_with_all_recs.append(user)
    # Be careful that for this case you have added a prerequisite 
    # that should be more than 2 ratings to be as an analyzed users
    else:
        users_with_part_recs.append(user)

users = np.unique(reviews['user_id'])
user_who_need_recs = np.setdiff1d(users, users_with_all_recs)
        
print('There are %s users who have at least 10 recommendations.' %len(users_with_all_recs))
print('%s users who have some but need more recommendations.' %len(users_with_part_recs))
print('So there are totally %s users who still need recommendations.' %len(user_who_need_recs))
print('Total users are %s.' %len(users))


There are 22187 users who have at least 10 recommendations.
1325 users who have some but need more recommendations.
So there are totally 45166 users who still need recommendations.
Total users are 67353.


### Content Based Recommendations

You will be doing a bit of a mix of content and collaborative filtering to make recommendations for the users this time.  This will allow you to obtain recommendations in many cases where we didn't make recommendations earlier.     

`2.` Before finding recommendations, rank the user's ratings from highest to lowest. You will move through the movies in this order looking for other similar movies.

In [4]:
# create a dataframe similar to reviews, but ranked by rating for each user
ranked_reviews = reviews.sort_values(by=['user_id', 'rating'], ascending=False)
ranked_reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020
863865,67353,1623205,6,1362832655,2013-03-09 12:37:35,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
863863,67352,172495,10,1587107015,2020-04-17 07:03:35,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
863864,67352,414387,10,1587107852,2020-04-17 07:17:32,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
863835,67351,5563334,7,1580591770,2020-02-01 21:16:10,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
863842,67351,6394270,7,1582666211,2020-02-25 21:30:11,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Similarities

In the collaborative filtering sections, you became quite familiar with different methods of determining the similarity (or distance) of two users.  We can perform similarities based on content in much the same way.  

In many cases, it turns out that one of the fastest ways we can find out how similar items are to one another (when our matrix isn't totally sparse like it was in the earlier section) is by simply using matrix multiplication.  If you are not familiar with this, an explanation is available [here by 3blue1brown](https://www.youtube.com/watch?v=LyGKycYT2v0) and another quick explanation is provided [in the post here](https://math.stackexchange.com/questions/689022/how-does-the-dot-product-determine-similarity).

For us to pull out a matrix that describes the movies in our dataframe in terms of content, we might just use the indicator variables related to **year** and **genre** for our movies.  

Then we can obtain a matrix of how similar movies are to one another by taking the dot product of this matrix with itself.  Notice below that the dot product where our 1 values overlap gives a value of 2 indicating higher similarity.  In the second dot product, the 1 values don't match up.  This leads to a dot product of 0 indicating lower similarity.

<img src="images/dotprod1.png" alt="Dot Product" height="500" width="500">

We can perform the dot product on a matrix of movies with content characteristics to provide a movie by movie matrix where each cell is an indication of how similar two movies are to one another.  In the below image, you can see that movies 1 and 8 are most similar, movies 2 and 8 are most similar, and movies 3 and 9 are most similar for this subset of the data.  The diagonal elements of the matrix will contain the similarity of a movie with itself, which will be the largest possible similarity (and will also be the number of 1's in the movie row within the orginal movie content matrix).

<img src="images/moviemat.png" alt="Dot Product" height="500" width="500">


`3.` Create a numpy array that is a matrix of indicator variables related to year (by century) and movie genres by movie.  Perform the dot product of this matrix with itself (transposed) to obtain a similarity matrix of each movie with every other movie.  The final matrix should be 31245 x 31245.

In [5]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Western,Adventure,Drama,...,Romance,Family,Biography,Musical,Sci-Fi,Thriller,Adult,Mystery,News,Game-Show
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Subset so movie_content is only using the dummy variables for each genre and the 3 century based year dummy columns
movie_content = np.array(movies.iloc[:, 4:])
# Take the dot product to obtain a movie x movie matrix of similaritiesA
dot_prod_movies = movie_content.dot(np.transpose(movie_content))
dot_prod_movies

array([[3, 3, 3, ..., 1, 1, 0],
       [3, 3, 3, ..., 1, 1, 0],
       [3, 3, 3, ..., 1, 1, 0],
       ..., 
       [1, 1, 1, ..., 2, 2, 1],
       [1, 1, 1, ..., 2, 2, 1],
       [0, 0, 0, ..., 1, 1, 1]])

In [7]:
dot_prod_movies.shape

(35479, 35479)

In [8]:
movies.shape

(35479, 35)

In [9]:
#Example:
a = np.array([[0, 1, 2],
              [0, 2, 4],
              [0, 3, 6]])
np.where(a < 4)
# if only gives condition, it return the index, 
# if gives both x and y, it can return the array values

# where inside where should refer to a list
# howevere it's dataframe for df.iloc(num/array) and df.loc(num/array)  

(array([0, 0, 0, 1, 1, 2, 2]), array([0, 1, 2, 0, 1, 0, 1]))

### For Each User...


Now you have a matrix where each user has their ratings ordered.  You also have a second matrix where movies are each axis, and the matrix entries are larger where the two movies are more similar and smaller where the two movies are dissimilar.  This matrix is a measure of content similarity. Therefore, it is time to get to the fun part.

For each user, we will perform the following:

    i. For each movie, find the movies that are most similar that the user hasn't seen.

    ii. Continue through the available, rated movies until 10 recommendations or until there are no additional movies.

As a final note, you may need to adjust the criteria for 'most similar' to obtain 10 recommendations.  As a first pass, I used only movies with the highest possible similarity to one another as similar enough to add as a recommendation.

`3.` In the cell below, complete each of the functions needed for making content based recommendations.

In [17]:
def find_similar_movies(movie_id):
    '''
    INPUT
        movie_id - a movie_id 
    OUTPUT
        similar_movies - an array of the most similar movies by title
    '''
    # find the row of each movie id
    movie_idx = np.where(movies['movie_id'] == movie_id)[0][0]
    
    # find the most similar movie indices - to start I said they need to be the same for all content
    similar_idxs = np.where(dot_prod_movies[movie_idx] == np.max(dot_prod_movies[movie_idx]))[0]
    
    # pull the movie titles based on the indices
    similar_movies = np.array(movies.iloc[similar_idxs, ]['movie'])
    
    return similar_movies
    
    
# You made this function in an earlier notebook - using again here    
def get_movie_names(movie_ids):
    '''
    INPUT
        movie_ids - a list of movie_ids
    OUTPUT
        movies - a list of movie names associated with the movie_ids
    
    '''
    movie_lst = list(movies[movies['movie_id'].isin(movie_ids)]['movie'])
   
    return movie_lst


def make_recs():
    '''
    INPUT
        None
    OUTPUT
        recs - a dictionary with keys of the user and values of the recommendations
    '''
    # Create dictionary to return with users and ratings
    recs = defaultdict(set)
    
    # How many users for progress bar
    num_users = len(users)
    
    # Create the progressbar
    bcounter = 0
    bar = progressbar.ProgressBar(maxval=num_users+1, widgets=[progressbar.Bar('=','[',']'), ' ', progressbar.Percentage()])
    bar.start()
    
    # For each user
    for user in users:
        
        # Update the progress bar
        bcounter+=1
        bar.update(bcounter)

        # Pull only the reviews the user has seen
        seen_movie_idxs_u = np.array(ranked_reviews[ranked_reviews['user_id'] == user]['movie_id'])
        seen_movie_names_u = get_movie_names(seen_movie_idxs_u)

        # Look at each of the movies (highest ranked first), 
        # pull the movies the user hasn't seen that are most similar
        # These will be the recommendations - continue until 10 recs 
        # or you have depleted the movie list for the user
        for m in seen_movie_idxs_u:
            recs_similar_m = find_similar_movies(m)
            recs_similar_m = np.setdiff1d(recs_similar_m, seen_movie_names_u)
            recs[user].update(recs_similar_m)
            
            if len(recs[user]) >= 10:
                break

    # finish the progress bar
    bar.finish()
    
    return recs

In [18]:
recs = make_recs()

[========================================================================] 100%


### How Did We Do?

Now that you have made the recommendations, how did we do in providing everyone with a set of recommendations?

`4.` Use the cells below to see how many individuals you were able to make recommendations for, as well as explore characteristics about individuals for whom you were not able to make recommendations.  

In [21]:
# Some characteristics of my content based recommendations
users_with_all_recs = [] # users with all recs here
users_without_all_recs = [] #store user ids without recs
no_recs = [] # users with no recommendations

for user, rec in recs.items():
    if len(rec) >= 10:
        users_with_all_recs.append(user)
    elif len(rec) != 0:
        users_without_all_recs.append(user)
    else:
        no_recs.append(user)

print("There were {} users with all 10 recommendations we would like them to have.".format(len(users_with_all_recs)))
print("There were {} users without all 10 recommendations we would have liked to have.".format(len(users_without_all_recs)))
print("There were {} users with no recommendations at all!".format(len(no_recs)))

There were 64779 users with all 10 recommendations we would like them to have.
There were 1969 users without all 10 recommendations we would have liked to have.
There were 605 users with no recommendations at all!


In [ ]:
# Closer look at individual user characteristics - this may help it was from an earlier notebook
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

def movies_watched(user_id):
    '''
    INPUT:
        user_id - the user_id of an individual as int
    OUTPUT:
        movies - an array of movies the user has watched
    '''
    movies = user_by_movie.loc[user_id][user_by_movie.loc[user_id].isnull() == False].index.values

    return movies


movies_watched(189)

In [ ]:
# More exploring
print("Some of the movie lists for users without any recommendations include:")

for user in no_recs[10]:
    print(user)
    print(get_movie_names(movies_watched(user)))

### Now What?  

Well, if you were really strict with your criteria for how similar two movies are (like I was initially), then you still have some users that don't have all 10 recommendations (and a small group of users who have no recommendations at all). 

As stated earlier, recommendation engines are a bit of an **art** and a **science**.  There are a number of things we still could look into - how do our collaborative filtering and content based recommendations compare to one another? How could we incorporate user input along with collaborative filtering and/or content based recommendations to improve any of our recommendations?  How can we truly gain recommendations for every user?

`5.` In this last step feel free to explore any last ideas you have with the recommendation techniques we have looked at so far.  You might choose to make the final needed recommendations using the first technique with just top ranked movies.  You might also loosen up the strictness in the similarity needed between movies.  Be creative and share your insights with your classmates!